In [1]:
import sys
sys.path.append('./yolo')
from detection_fusion import detection_fusion
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from rcnn.dataset import ChestCocoDetection
from torchvision import transforms
import matplotlib.patches as patches
from torch.utils.data import DataLoader
import tqdm

In [2]:
def add_bounding_boxes(target, ax):
    boxes = target['boxes']
    for box in boxes:
        mp_box = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], edgecolor="r", facecolor='none')
        ax.add_patch(mp_box)

def show_samples_for(test, train):
    fig, axs = plt.subplots(1, 2, figsize=(10,8))
    #fig.suptitle(f'Study: {samples["id"].iloc[i]}')
    for im_i, (img, data) in enumerate([test, train]):
        ax = axs[im_i] if isinstance(axs, np.ndarray) else axs
        ax.set_title(f'Instance / Image Nr. {im_i + 1} / {2}')
        #dcm = pydicom.dcmread(image_path)
        plt.figure()
        ax.imshow(img.permute(1,2,0), cmap=plt.cm.bone)
        add_bounding_boxes(data, ax)
    fig.savefig("combo.jpg")

In [3]:
test = ChestCocoDetection(root="F:\\aml-project\data\\siim-covid19-detection", ann_file="F:\\aml-project\\data\\siim-covid19-detection\\test.json", training=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
idx = np.random.randint(len(test))
test_img = test.__getitem__(idx)
print(type(test_img[0]), test_img[0].shape)
img = test_img[0]
img_pil = transforms.ToPILImage()(img)

<class 'torch.Tensor'> torch.Size([3, 1024, 1024])


In [5]:
results, boxes, scores = detection_fusion(img_pil)

D:\Anaconda\envs\aml\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [6]:
show_samples_for(test_img, (test_img[0], {"boxes":boxes}))

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


### Evaluation of Ensemble method

In [7]:
def collate_fn(batch):
    imgs, targets = zip(*batch)
    imgs = torch.stack(imgs)
    return tuple((imgs, targets))

In [8]:
# need bs = 1 because model fusion cannot handle batches 
test_loader = DataLoader(test, batch_size=1, shuffle=False, pin_memory=False, num_workers=1, collate_fn=collate_fn)

In [9]:
jdict, stats, ap, ap_class = [], [], [], []
for images, targets in tqdm.tqdm(test_loader):
    nb, _, height, width = images.shape
    assert nb == 1
    img_pil = transforms.ToPILImage()(img)[0]
    _, boxes, scores, labels, _, _, _, _ = detection_fusion(img_pil)
    
     # Statistics per image
    for si, pred in enumerate(out):
    labels = targets[targets[:, 0] == si, 1:]
    nl = len(labels)
    tcls = labels[:, 0].tolist() if nl else []  # target class
    path = Path(paths[si])
    seen += 1

    if len(pred) == 0:
        if nl:
            stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
        continue

    # Predictions
    pred[:, 5] = 0
    predn = pred.clone()
    scale_coords(imgs[si].shape[1:], predn[:, :4], shapes[si][0], shapes[si][1])


    correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool, device=device)
    if nl:
        detected = []
        tcls_tensor = labels[:, 0]

        # boxes
        tbox = xywh2xyxy(labels[:, 1:5])
        scale_coords(imgs[si].shape[1:], tbox, shapes[si][0], shapes[si][1])

        # Per target class
        for cls in torch.unique(tcls_tensor):
            ti = (cls == tcls_tensor).nonzero(as_tuple=False).view(-1)
            pi = (cls == pred[:, 5]).nonzero(as_tuple=False).view(-1) 

            # Search for detections
            if pi.shape[0]:
                ious, i = box_iou(predn[pi, :4], tbox[ti]).max(1)  # best ious, indices

                # Append detections
                detected_set = set()
                for j in (ious > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i[j]]  
                    if d.item() not in detected_set:
                        detected_set.add(d.item())
                        detected.append(d)
                        correct[pi[j]] = ious[j] > iouv  
                        if len(detected) == nl: 
                            break

        # Append statistics (correct, conf, pcls, tcls)
        stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))

stats = [np.concatenate(x, 0) for x in zip(*stats)] 
if len(stats) and stats[0].any():
    p, r, ap, f1, ap_class = ap_per_class(*stats)
    ap50, ap = ap[:, 0], ap.mean(1) 
    mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
    nt = np.bincount(stats[3].astype(np.int64), minlength=NUMBER_OF_CLASSES)

    general_test_results.append({
    "precision": p,
    "recall": r,
    "ap": ap,
    "f1": f1,
    "ap_class": ap_class,
    "ap": ap,
    "ap50": ap50,
    "mp": mp,
    "mr": mr,
    "map50": map50,
    "map": map
})

else:
    nt = torch.zeros(1)

# Print results
pf = '%20s' + '%12i' * 2 + '%12.3g' * 4 
print(pf % ('all', seen, nt.sum(), mp, mr, map50, map))

IndentationError: expected an indented block (Temp/ipykernel_8244/631558422.py, line 10)